In [7]:
import pandas as pd 
import numpy as np 
import os 

In [9]:
anime = pd.read_csv("../dataset/anime.csv") 
ratings = pd.read_csv("../dataset/rating.csv") 

In [10]:
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [11]:
ratings

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [18]:
strange_ratings = ratings.loc[ratings.rating < 0, :]

In [27]:
anime[anime.name.str.contains("Gintama")] 

,anime_id,name,genre,type,episodes,rating,members
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201,9.04,336376
41,32366,Gintama°: Aizome Kaori-hen,"Comedy, Parody",OVA,2,8.69,16947
63,21899,Gintama: Yorinuki Gintama-san on Theater 2D,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,2,8.60,11104
65,7472,Gintama Movie: Shinyaku Benizakura-hen,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,8.59,51185
171,6945,Gintama: Shiroyasha Koutan,"Action, Comedy, Historical, Parody, Sci-Fi",Special,1,8.37,27213
216,9735,Gintama: Shinyaku Benizakura-hen,"Action, Comedy, Historical, Parody, Samurai, S...",Special,1,8.31,14002


In [39]:
# getting the list of all genre's that exist inside the python file 
genres = set() 
genre_lists = anime.genre.unique() 
for genre_list in genre_lists: 
    if isinstance(genre_list, str): 
        genre_items = genre_list.split(",")
        for genre in genre_items: 
            genres.add(genre.strip())
genres = list(genres) 


In [48]:
from typing import List, Dict 

def generate_encodings(genre_string: str, genres: List[str]) -> np.array: 
    vector: np.array = np.zeros((1, len(genres)))  
    
    if isinstance(genre_string, str): 
        genre_map: Dict[str, int] = { gn: idx for idx, gn in enumerate(genres) } 
        genre_list: List[str] = genre_string.split(",") 
        for genre in genre_list:
                vector[0, genre_map[genre.strip()]] = 1  
                
    return vector 

vectors = anime.genre.apply(lambda x: generate_encodings(x, genres))  

In [57]:
genre_encodings = np.vstack(vectors.values.tolist()).astype(np.int32) 

In [77]:
type_disintegration = pd.get_dummies(anime["type"], drop_first=True) 

In [63]:
genre_map: Dict[str, int] = { gn: idx for idx, gn in enumerate(genres) } 

In [70]:
anime_map: Dict[int, str] = {} 
for _, anime_row in anime.iterrows(): 
    anime_id = anime_row["anime_id"] 
    anime_name = anime_row["name"] 
    anime_map[anime_id] = anime_name 


In [72]:
items = genre_map.items() 
sorted(items, key = lambda x: x[1])  
genre_names = [name for name, _ in items] 
encoding_df = pd.DataFrame(genre_encodings,columns=genre_names ) 

In [74]:
encoding_df = encoding_df.drop(columns=["Hentai"]) 

In [78]:
processed_anime = pd.concat([
    anime[["anime_id", "episodes", "rating", "members"]], 
    type_disintegration, encoding_df], axis=1) 

In [82]:
processed_anime = processed_anime.set_index("anime_id") 

In [83]:
processed_anime

,episodes,rating,members,Music,ONA,OVA,Special,TV,Slice of Life,Psychological,...,Comedy,Super Power,Martial Arts,Shoujo,Dementia,Music,Josei,Sports,Samurai,Romance
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,1,9.37,200630,False,False,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,1
5114,64,9.26,793665,False,False,False,False,True,0,0,...,0,0,0,0,0,0,0,0,0,0
28977,51,9.25,114262,False,False,False,False,True,0,0,...,1,0,0,0,0,0,0,0,1,0
9253,24,9.17,673572,False,False,False,False,True,0,0,...,0,0,0,0,0,0,0,0,0,0
9969,51,9.16,151266,False,False,False,False,True,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,1,4.15,211,False,False,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
5543,1,4.28,183,False,False,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
5621,4,4.88,219,False,False,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import MinMaxScaler, StandardScaler 